# Plan

- intro: why tidy data
- getting to tidy data (quick overview, which is tailored based on Jacobo's materials)
- split-apply-combine
- seaborn

In [2]:
import pandas as pd
print(pd.__version__)

import numpy as np

0.20.3


# Tidy data

The concept of *tidy data* was introduced by Hadley Wickham (Chief scientist in **R** project) and was inspired by databases. It turns out that scientists and statisticians can benifit very much from the same concepts. In particular, structuring your data in a tidy way will facilitate any type of analysis you want to do.

The core ideas are taken from Hadley Wickham's seminal paper "Tidy data" [1].

[1] Wickham, H. (2014). Tidy data. Journal of Statistical Software, 59(10), 1-23.

# Tidying: structuring datasets to facilitate analysis

The principles of tidy data provide a standard way to organize data values within a dataset. Current tools often require translation: you have to spend time munging the output from one tool so you can input it into another. Tidy datasets and tools for them work hand in hand to make data analysis easier, allowing you to focus on the interesting domain problem, not on the uninteresting logistics of data.

Let's start with an example.

Consider the following 2 ways of presenting the same toy data. Think about whether there is any difference how we organize it.

In [1]:
untidy = pd.DataFrame({'treatment_a':[np.nan, 16, 3],'treatment_b':[2,11,1]}, 
                      index=['John Smith', 'Jane Doe','Mary Johnson'])
untidy

NameError: name 'pd' is not defined